#MNIST 예시

In [10]:
import torch
# 신경망 구축에 필요한 여러 매서드를 담은 torch.nn
import torch.nn as nn
# torch.nn의 모든 함수를 포함, 손실/활성화/풀링/합성곱/선형 및 기타 신경망 함수 가 포함됨
import torch.nn.functional as F
# 최적화(신경망의 가중치,매개변수 조정을 위해 오차를 역전파 하는 과정) 모듈이 담김
import torch.optim as optim

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [11]:
class ConvNet(nn.Module):
  """
  kernel_size : 보통 홀수. 너무 작으면 픽셀을 처리하는 kernel이 이웃 픽셀의 정보를 가지지 못한다.너무 크면 이미지 내에서 정밀하지
                않은 특징을 얻게 됨. 작은 kernel_size의 많은 layer를 쓰면 네트워크가 깊어지고, 더 복잡한 특징 학습 가능.

  feature map : 이미지 데이터에서 픽셀 정보를 담고 있는 channel(차원)을 의미. 이미지에서 더 많은 특징을 추출하려거든 channel을 크게
                하면 된다.

  input shape이 28x28x1임

  Conv2d(input_channel, output_channel, kernel_size, stride)
  """
  # 각 layer의 뉴런개수 및 layer들 정의
  def __init__(self):
    super(ConvNet, self).__init__()

    # 합성곱 layer
    self.cn1 = nn.Conv2d(1,16,3,1)
    self.cn2 = nn.Conv2d(16,32,3,1)
    # 드롭아웃 layer
    self.dp1 = nn.Dropout(0.1)
    self.dp2 = nn.Dropout(0.25)
    # fully-connected(fc) layer, 4608=12x12x32
    self.fc1 = nn.Linear(4608, 64)
    # 최종 출력은 10개 클래스 중 하나
    self.fc2 = nn.Linear(64,10)

  def forward(self, x):
    x = self.cn1(x)
    x = F.relu(x)
    x = self.cn2(x)
    x = F.relu(x)
    # kernel size가 2x2
    x = F.max_pool2d(x,2)
    x = self.dp1(x)
    # 1차원 백터로 평면화
    x = torch.flatten(x,1)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.dp2(x)
    x = self.fc2(x)
    # 모델의 예측을 out에 담음
    out = F.log_softmax(x, dim=1)
    return out

In [21]:
# 훈련 루틴 정의
def train(model, device, train_dataloader, optim, epoch):
  # 모델 훈련
  model.train()
  # batch 단위로 반복
  for b_i, (X,y) in enumerate(train_dataloader):
    # 주어진 로컬 메모리에 데이터셋 사본 만듦
    X,y = X.to(device), y.to(device)
    # 이전에 계산했던 gradient를 초기화(이전 값들은 이미 이전단계의 파라미터 수정할 때 쓰였으니까 노필요)
    optim.zero_grad()
    # 주어진 입력데이터를 활용하여 모델 예측 실행
    pred = model(X)
    # negative log liklihood, 모델예측값과 실제값 사이의 손실 계산
    loss = F.nll_loss(pred, y)
    # 역전파, 자동미분됨
    loss.backward()
    # 가중치 조정
    optim.step()
    if b_i % 10 == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, b_i * len(train_dataloader), len(train_dataloader.dataset),100. * b_i / len(train_dataloader), loss.item()))

In [23]:
# 테스트 루틴 정의
def test(model, device, test_dataloader):
  """
  torch.no_grad : 추론(평가) 과정에서 사용하는 autograd 끄는 함수. 모델이 파라미터 업데이트 안 하고(즉, 학습X) 단순히 예측/추론만 함.

  평균적인 손실(오차) 정도를 구하기 위해 loss를 합한다.
  """

  # 모델 성능 평가
  model.eval()
  loss = 0
  success = 0
  with torch.no_grad():
    for X, y in test_dataloader:
      X,y = X.to(device), y.to(device)
      pred = model(X)
      # 배치별 손실의 합(옵티마이저X->모델 가중치 조정X->모델평가를 위해 배치 단위로 오차 합함)
      loss += F.nll_loss(pred, y, reduction='sum').item()
      pred = pred.argmax(dim=1, keepdim=True)
      success += pred.eq(y.view_as(pred)).sum().item()
    loss /= len(test_dataloader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(loss, success, len(test_dataloader.dataset),100. * success / len(test_dataloader.dataset)))

In [19]:
"""
torch.DataLoader : DataLoader에 입력되는 dataset을 모델에 배치입력하기 쉽게 만들어주는 모듈(추가적인 동작이 필요할 경우 DataSet을 상속받는
                   사용자 정의 DataSet을 만들어 그 DataSet을 DataLoader에 넣기도 함). 그래서 Dataset갹채에서 배치단위 데이터를 가져온 걸 반환.

transform 파리미터는 MNIST 이미지 데이터에 대한 전처리 과정을 정의.

transforms.Compose() 괄호 안에 들어가는 변환함수들을 담은 리스트는 순차적으로 입력데이터에 적용됨

transforms.Normalize((mean, std))

batch_size : 한번에 모델에 입력되는 데이터 묶음. 총 1000개의 이미지데이터가 있다면 batch_size가 32이라면 한 번 학습 시에
             32개의 이미지(데이터,혹은 샘플)가 쓰인다는 것. 그리고 이때 1000/32 약 32개의 배치(데이터묶음)가 생기는데
             마지막 배치의 경우 (1000-31)*32로 8개의 이미지데이터가 담김
# # 훈련 데이터 불러오기
# train_dataloader = torch.utils.data.DataLoader(datasets.MNIST("../data", train=True, download=True, transform=transforms.Compose([transforms.ToTensor(),
#                                                                                                                                   transforms.ToTensor(),
#                                                                                                                                   transforms.Normalize((0.1302,),(0.3069))])),
#                                                batch_size=32, shuffle=True)
# # 성능 평가를 위한 데이터 불러오기
# test_dataloader = torch.utils.data.DataLoader(datasets.MNIST("../data", train=False, transform=transforms.Compose([transforms.ToTensor(),
#                                                                                                                    transforms.Normalize((0.1302,),(0.3069))])),
#                                                batch_size=500, shuffle=True)
"""

transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                    transform=transform)
train_dataloader = torch.utils.data.DataLoader(dataset1,batch_size=32, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(dataset2, batch_size=500, shuffle=True)

In [15]:
# 무작위성X/재현가능성 위해 시드값 설정
torch.manual_seed(0)
# 연산 수행할 장치 지정
device = torch.device("cpu")
# 모델 객체 생성
model = ConvNet()
# 옵티마이저 객체 생성
optimizer = optim.Adadelta(model.parameters(), lr=0.5)

In [24]:
# 모델을 실제로 훈련,테스트
# %capture
for epoch in range(1,3):
  train(model, device, train_dataloader, optimizer, epoch)
  test(model, device, test_dataloader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.362310
Train Epoch: 1 [18750/60000 (1%)]	Loss: 0.093845
Train Epoch: 1 [37500/60000 (1%)]	Loss: 0.220424
Train Epoch: 1 [56250/60000 (2%)]	Loss: 0.221767
Train Epoch: 1 [75000/60000 (2%)]	Loss: 0.007746
Train Epoch: 1 [93750/60000 (3%)]	Loss: 0.042532
Train Epoch: 1 [112500/60000 (3%)]	Loss: 0.301133
Train Epoch: 1 [131250/60000 (4%)]	Loss: 0.080663
Train Epoch: 1 [150000/60000 (4%)]	Loss: 0.179335
Train Epoch: 1 [168750/60000 (5%)]	Loss: 0.082491
Train Epoch: 1 [187500/60000 (5%)]	Loss: 0.104114
Train Epoch: 1 [206250/60000 (6%)]	Loss: 0.186040
Train Epoch: 1 [225000/60000 (6%)]	Loss: 0.058609
Train Epoch: 1 [243750/60000 (7%)]	Loss: 0.076216
Train Epoch: 1 [262500/60000 (7%)]	Loss: 0.015135
Train Epoch: 1 [281250/60000 (8%)]	Loss: 0.077796
Train Epoch: 1 [300000/60000 (9%)]	Loss: 0.016076
Train Epoch: 1 [318750/60000 (9%)]	Loss: 0.004198
Train Epoch: 1 [337500/60000 (10%)]	Loss: 0.002653
Train Epoch: 1 [356250/60000 (10%)]	Loss: 0.048009
Train Ep

In [25]:
# 모델을 훈련 후 테스트셋을 통해 성능도 검증했으니 샘플 이미지에서 추론이 맞는지 확인
test_samples = enumerate(test_dataloader)
b_i, (sample_data, sample_targets) = next(test_samples)

# 확률이 가장 높은 클래스(max)를 선택. model(sample_data)[1]은 숫자 분류 결과 배열.
print(f"model prediction : {model(sample_data).data.max(1)[1][0]}")
print(f"Ground truth : {sample_targets[0]}")

model prediction : 0
Ground truth : 0


#CNN과 LSTM 결합하기